In [1]:
import dedalus.public as d3
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# How to write d3 data to disk for initial conditions

First, create the coordinate system, bases, and distributor as you would in the script.

In [2]:
dtype = np.float64
dealias = 3/2
nx = 32
ny = 32
nz =32
Lx = 5
Ly = 5

In [3]:
coords = d3.CartesianCoordinates('x', 'y', 'z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=nx, bounds=(0, Lx), dealias = dealias)
ybasis = d3.RealFourier(coords['y'], size=ny, bounds=(0, Ly), dealias = dealias)
zbasis = d3.ChebyshevT(coords['z'], size=nz, bounds = (-1,1), dealias = dealias)
x = xbasis.local_grid(1)
y = xbasis.local_grid(1)
z = zbasis.local_grid(1)

Make sure to create *all* the fields you need to run.

In [4]:
ba = (xbasis,ybasis,zbasis)
ba_p = (xbasis,ybasis)

p = dist.Field(name='p', bases=ba)
u = dist.VectorField(coords, name='u', bases=ba)
tau_u1 = dist.VectorField(coords, name='tau_u1', bases=ba_p)
tau_u2 = dist.VectorField(coords, name='tau_u2', bases=ba_p)
tau_p = dist.Field(name='tau_p')


In [5]:
u['g'].shape

(3, 32, 32, 32)

Add something to `u`; you can leave everything else as zeros.

Next, you'll create an `Evaluator` object with the distributor `dist` and the dictionary of fields. You won't need to modify any of this code below.

In [6]:
from dedalus.core.evaluator import Evaluator

In [7]:
evaluator = Evaluator(dist,{'p':p, 'u':u})

In [9]:
check = evaluator.add_file_handler(Path('checkpoints'), iter=10, max_writes=1, virtual_file=True)
check.add_tasks([u,p,tau_u1, tau_u2, tau_p])


Then you'll need to call `evaluator.evaluate_handlers([check])`, which should write a `checkpoints/checkpoints_s1` directory. You'll need the entire contents of that directory, but you will restart from the file `checkpoints/checkpoints_s1/checkpoints_s1.h5`

In [10]:
evaluator.evaluate_handlers([check])